<a href="https://colab.research.google.com/github/kaodamie/PAN-CLEF---Profilling-Hate-Spreaders-on-Twiter/blob/main/BertExperiment2_eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -q tensorflow-text


In [ ]:
pip install -q tf-models-official

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import os 
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as mpl
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow import feature_column
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import shutil
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization 
tf.get_logger().setLevel('ERROR')

name=r'/content/drive/MyDrive/Dataset/pan21-author-profiling-training-2021-03-14/en//'

xml_text=[]
list_filenames=[]
for file in os.listdir(name):
        filename = os.fsdecode(file)
        
        if filename.endswith('.xml'):
#             print(filename)   
            list_filenames.append(filename.split(sep='.')[0])
            try:
                tree = ET.parse(name+file) 
                root =tree.getroot() 
                xml_document=[]
                for neighbor in root.iter('document'):
                    xml_document.append(neighbor.text) #neighbor.text.lower()
                     
                    #print(xml_document)
#                 print('\n\n')
                xml_text.append(" ".join(xml_document))
            except Exception as e:
                print(e, name+file)
#                 display(e, folder+file)
        
        elif filename.endswith('txt'):
            fileObject = open(name+file)
            data = fileObject.read().splitlines()
            dat=[]
            for i in data:
                dat.append(i.split(sep=':::'))
                
                
# (list_filenames,xml_text,dat)
# es_xml_text.append(xml_text)

In [ ]:
# join the labels to the text
df1=pd.DataFrame(zip(list_filenames,xml_text),columns=('ids','xml_text')).set_index('ids').join(pd.DataFrame(dat,columns=('ids','labels')).set_index(['ids']))
# v=pd.DataFrame(dat)
# v['xml'] =xml_text


# in future make a k-fold cross validation with sklearn
train, test = train_test_split(df1, test_size=0.2)
val, test = train_test_split(test, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

train_ds=train['xml_text'].astype(str).to_numpy()
val_ds=val['xml_text'].astype(str).to_numpy()

whole_dataset=df1['xml_text'].astype(str).to_numpy()
whole_labels=df1['labels'].astype(int).to_numpy()

train_labels=train['labels'].astype(int).to_numpy()
val_labels=val['labels'].astype(int).to_numpy()

In [ ]:
df1['xml_text']


In [ ]:
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

train_ds = tf.data.Dataset.from_tensor_slices((train_ds,train_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((val_ds,val_labels))

whole_ds=tf.data.Dataset.from_tensor_slices((whole_dataset,whole_labels))

train_ds = train_ds.batch(BATCH_SIZE)
valid_ds = valid_ds.batch(BATCH_SIZE)

# PREFETCH

train_ds = train_ds.shuffle(5).prefetch(buffer_size=AUTOTUNE)
valid_ds = valid_ds.shuffle(5).prefetch(buffer_size=AUTOTUNE)
# whole_ds=whole_ds.shuffle(5).prefetch(buffer_size=AUTOTUNE)

In [ ]:
bert_model_name = 'albert_en_base' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

In [ ]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

text_test = 'this is such an amazing movie!'
# tt=' '.join(text_test)
text_preprocessed = bert_preprocess_model([text_test])

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

In [ ]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [ ]:
# best model so far

def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [ ]:
# def build_classifier_model_1():
#   text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
#   preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
#   encoder_inputs = preprocessing_layer(text_input)
#   encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
#   outputs = encoder(encoder_inputs)
#   net = outputs['pooled_output']
#   net = tf.keras.layers.Dropout(0.25)(net)
#   net = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(net)
#   return tf.keras.Model(text_input, net)

In [ ]:
# def build_classifier_model_2():
#   text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
#   preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
#   encoder_inputs = preprocessing_layer(text_input)
#   encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
#   outputs = encoder(encoder_inputs)
#   net = outputs['pooled_output']
#   net = tf.keras.layers.Dropout(0.1)(net)
#   net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
#   return tf.keras.Model(text_input, net)

# classifier_model_2= build_classifier_model_2()  

In [ ]:
classifier_model = build_classifier_model()
# classifier_model_1= build_classifier_model_1()
# bert_raw_result = classifier_model(tf.constant(text_test))
# print(tf.sigmoid(bert_raw_result))

In [ ]:
tf.keras.utils.plot_model(classifier_model)

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [ ]:


epochs = 50
steps_per_epoch =  tf.data.experimental.cardinality(train_ds).numpy()
# tf.data.experimental.cardinality(np.array(train_ds)).numpy()
# len(train)
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 1e-6
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

# monitoring the val_loss is the best way as monitoring actual training los results in overfitting ..
mcp_save = ModelCheckpoint('bert.h5', save_best_only=True, monitor='val_loss', mode='min')

early_stop=EarlyStopping(monitor='val_loss', patience=5, verbose=0)

In [ ]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [ ]:
# print(f'Training model with {tfhub_handle_encoder}')
# history = classifier_model_2.fit(train_ds,
#                                validation_data=valid_ds,
#                                epochs=epochs,callbacks=[mcp_save, early_stop], verbose=1)

In [ ]:
# Keep!
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(train_ds,
                               validation_data=valid_ds,
                               epochs=epochs,callbacks=[mcp_save, early_stop], verbose=1)

In [ ]:
classifier_model.save('/content/drive/MyDrive/Dataset/PanResults/es')

In [ ]:
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(train_ds,
                               validation_data=valid_ds,
                               epochs=epochs,callbacks=[mcp_save, early_stop], verbose=0)

In [ ]:
# output_model=tf.keras.models.load_model("bert.h5", compile)

# classifier_model.load_weights('bert.h5')

In [ ]:
train_loss, train_acc = classifier_model.evaluate(train_ds, verbose=1)
validation_loss, validation_acc =classifier_model.evaluate(valid_ds, verbose=1)
#whole_loss, whole_acc = classifier_model.evaluate(whole_ds, verbose=0)
# test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=1)

print('* Accuracy on training set: %0.2f%%' % (100 * train_acc))
print('* Accuracy on validation set: %0.2f%%' % (100 * validation_acc))
#print('* Accuracy on whole set: %0.2f%%' % (100 * whole_acc))

In [ ]:
reloaded=tf.saved_model.load('/content/drive/MyDrive/Dataset/PanResults/en/savedmodel')

In [ ]:
results=tf.sigmoid(reloaded(tf.constant(test['xml_text'])))
#test_results=round()

In [ ]:
test['results_1']=results.numpy()

In [ ]:
round(results[0][0].numpy())

In [ ]:
test

In [ ]:
#working with the actual test data
name=r'/content/drive/MyDrive/Dataset/pan21-author-profiling-test-without-gold/en//'

xml_text=[]
list_filenames=[]
for file in os.listdir(name):
        filename = os.fsdecode(file)
        
        if filename.endswith('.xml'):
#             print(filename)   
            list_filenames.append(filename.split(sep='.')[0])
            try:
                tree = ET.parse(name+file) 
                root =tree.getroot() 
                xml_document=[]
                for neighbor in root.iter('document'):
                    xml_document.append(neighbor.text)
                     
                    #print(xml_document)
#                 print('\n\n')
                xml_text.append(" ".join(xml_document))
            except Exception as e:
                print(e, name+file)
#                 display(e, folder+file)

In [ ]:
df2=pd.DataFrame(zip(list_filenames,xml_text),columns=('ids','xml_text')).set_index('ids')

In [ ]:
df2


In [ ]:
test_results=tf.sigmoid(reloaded(tf.constant(df2['xml_text'])))

In [ ]:

for i in range(len(df2['xml_text'])):
  # print(f'{df1.index[i]}: {round(results[i][0].numpy())}')
  root = ET.Element('author', id=df2.index[i],lang='en',type=f'{round(test_results[i][0].numpy())}')
  s = ET.tostring(root)
  print(str(s)) 
  tree = ET.ElementTree(root)
  tree.write("/content/drive/MyDrive/Dataset/PanResults/Test_Results/en/%s.xml" %df2.index[i])